In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import torch
import hydra
from pathlib import Path
from lightning import Fabric

# 设置 torch.compile 兼容性
try:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
except ImportError:
    # PyTorch 版本 < 2.0 不支持 torch._dynamo
    print("Warning: torch._dynamo not available in this PyTorch version")

## set up environment
# 使用绝对路径指向项目根目录
project_root = "/datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield"
sys.path.insert(0, str(project_root))

from funcmol.utils.constants import PADDING_INDEX
from funcmol.gnf_visualizer import (
    load_config_from_exp_dir, load_model, 
    create_converter, prepare_data, visualize_1d_gradient_field_comparison,
    GNFVisualizer
)

# 模型根目录
model_root = "/datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field"

wandb: WARNING This integration is tested and supported for lightning Fabric 2.1.3.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `lightning-fabric`.
Fabric will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Fabric(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


In [2]:
# TODO：只需要修改这里的就好，会根据exp_name判断gradient_field_method
exp_name = 'nf_drugs_20250819_100445_894697'  # 使用drugs实验
sample_idx = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 强制设置为 gt_only 模式，但仍然需要实验目录来获取配置
option = 'gt_only'
model_dir = os.path.join(model_root, exp_name)  # 使用实验目录来获取配置文件

output_dir = model_dir
print(f"Option: {option}")
print(f"Model directory: {model_dir}")
print(f"Output directory: {output_dir}")

Option: gt_only
Model directory: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697
Output directory: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697


In [ ]:
## Load data
fabric = Fabric(
    accelerator="auto",
    devices=1,
    precision="32-true",
    strategy="auto"
)
fabric.launch()

# 从实验目录加载配置
config = load_config_from_exp_dir(model_dir)

# 准备数据
batch, gt_coords, gt_types = prepare_data(fabric, config, device)
print(f"Data loaded: {gt_coords.shape}, {gt_types.shape}")

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Dataset directory: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/funcmol/dataset/data


In [ ]:
print(f"\nProcessing model from: {model_dir}")

converter = create_converter(config, device)
## Load model
if option == 'gt_pred':
    encoder, decoder = load_model(fabric, config, model_dir=model_dir)
    # 生成 codes
    with torch.no_grad():
        codes = encoder(batch)
    # 定义预测场函数
    def predicted_field_func(points):
        # 确保 points 是正确的形状
        if points.dim() == 2:  # [n_points, 3]
            points = points.unsqueeze(0)  # [1, n_points, 3]
        elif points.dim() == 3:  # [batch, n_points, 3]
            pass
        else:
            raise ValueError(f"Unexpected points shape: {points.shape}")
        
        result = decoder(points, codes[sample_idx:sample_idx+1])
        # 确保返回 [n_points, n_atom_types, 3] 形状
        if result.dim() == 4:  # [batch, n_points, n_atom_types, 3]
            return result[0]  # 取第一个batch
        else:
            return result
    field_func = predicted_field_func
else:  # gt only
    encoder, decoder = None, None
    # 定义真实场函数
    def gt_field_func(points):
        gt_mask = (gt_types[sample_idx] != PADDING_INDEX)
        gt_valid_coords = gt_coords[sample_idx][gt_mask]
        gt_valid_types = gt_types[sample_idx][gt_mask]
        
        # 确保 points 是正确的形状
        if points.dim() == 2:  # [n_points, 3]
            points = points.unsqueeze(0)  # [1, n_points, 3]
        elif points.dim() == 3:  # [batch, n_points, 3]
            pass
        else:
            raise ValueError(f"Unexpected points shape: {points.shape}")
        
        result = converter.mol2gnf(
            gt_valid_coords.unsqueeze(0),
            gt_valid_types.unsqueeze(0),
            points
        )
        # 确保返回 [n_points, n_atom_types, 3] 形状
        if result.dim() == 4:  # [batch, n_points, n_atom_types, 3]
            return result[0]  # 取第一个batch
        else:
            return result
    field_func = gt_field_func

print(f"Model loaded successfully!")


Processing model from: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697
GNF Converter created with n_iter: 3000, gradient_field_method: tanh, n_atom_types: 8
Model loaded successfully!


In [ ]:
# 可视化一维梯度场对比（所有原子类型）
# drugs数据集包含8种原子类型: C, H, O, N, F, S, Cl, Br
atom_types = [0, 1, 2, 3, 4, 5, 6, 7]  # C, H, O, N, F, S, Cl, Br
save_path = os.path.join(output_dir, "recon", f"field1d_sample_{sample_idx}")

gradient_results = visualize_1d_gradient_field_comparison(
    gt_coords=gt_coords,
    gt_types=gt_types,
    converter=converter,
    field_func=field_func,
    sample_idx=sample_idx,
    atom_types=atom_types,  # 传入列表，不需要循环
    x_range=None,
    y_coord=0.0,
    z_coord=0.0,
    save_path=save_path,
)

if gradient_results:
    print(f"Gradient field comparison (model: {model_dir}):")
    print(f"  Available atom types: {gradient_results['available_atom_types']}")
    
    # 打印每个原子类型的统计信息
    for atom_name, stats in gradient_results['all_results'].items():
        print(f"  {atom_name}: MSE={stats['mse']:.6f}, MAE={stats['mae']:.6f}")
        print(f"    Saved to: {stats['save_path']}")

警告：样本 1 中没有类型为 O 的原子
警告：样本 1 中没有类型为 F 的原子
警告：样本 1 中没有类型为 Cl 的原子
自动计算 x 轴范围: (-7.120750427246094, 7.120750427246094)
Field 1D comparison (atom_type=C) saved to: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/field_1d_sample_1_atom_C.png
Field 1D comparison (atom_type=H) saved to: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/field_1d_sample_1_atom_H.png
Field 1D comparison (atom_type=N) saved to: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/field_1d_sample_1_atom_N.png
Field 1D comparison (atom_type=S) saved to: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/field_1d_sample_1_atom_S.png
Field 1D comparison (atom_type=Br) saved to: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_2

In [ ]:
# 根据option设置重建列表
if option == 'gt_only':
    rec_list = ['gt_field']
else:
    rec_list = ['predicted_field', 'gt_field']

# 创建可视化器
visualizer = GNFVisualizer(output_dir)

# 为每种重建类型执行可视化
for rec_type in rec_list:
    print(f"\n=== 执行 {rec_type} 重建 ===")
    
    # 根据重建类型设置场函数
    if rec_type == 'gt_field':
        # 定义真实场函数
        def gt_field_func(points):
            gt_mask = (gt_types[sample_idx] != PADDING_INDEX)
            gt_valid_coords = gt_coords[sample_idx][gt_mask]
            gt_valid_types = gt_types[sample_idx][gt_mask]
            return converter.mol2gnf(
                gt_valid_coords.unsqueeze(0),
                gt_valid_types.unsqueeze(0),
                points
            )
        field_func = gt_field_func
    else:  # predicted_field
        # 定义预测场函数
        def predicted_field_func(points):
            if points.dim() == 2:
                points = points.unsqueeze(0)
            elif points.dim() == 3:
                pass
            else:
                raise ValueError(f"Unexpected points shape: {points.shape}")
            result = decoder(points, codes[sample_idx:sample_idx+1])
            return result[0] if result.dim() == 4 else result
        field_func = predicted_field_func
    
    # 执行重建可视化
    results = visualizer.create_reconstruction_animation(
        gt_coords=gt_coords,
        gt_types=gt_types,
        converter=converter,
        field_func=field_func,
        save_interval=100,
        animation_name=f"recon_sample_{sample_idx}_{rec_type}",
        sample_idx=sample_idx
    )

    print(f"\n=== {rec_type} 重建结果 ===")
    print(f"RMSD: {results['final_rmsd']:.4f}")
    print(f"Reconstruction Loss: {results['final_loss']:.4f}")
    print(f"KL Divergence (orig->recon): {results['final_kl_1to2']:.4f}")
    print(f"KL Divergence (recon->orig): {results['final_kl_2to1']:.4f}")
    print(f"GIF动画: {results['gif_path']}")
    print(f"对比图: {results['comparison_path']}")


=== 执行 gt_field 重建 ===

Starting reconstruction for molecule 1
Ground truth atoms: 34
[DBSCAN] Total points: 1000, Clusters found: 13, Noise points: 87
[DBSCAN] Total points: 1000, Clusters found: 11, Noise points: 29
[DBSCAN] Total points: 1000, Clusters found: 0, Noise points: 1000
[DBSCAN] Total points: 1000, Clusters found: 6, Noise points: 238
[DBSCAN] Total points: 1000, Clusters found: 0, Noise points: 1000

=== gt_field 重建结果 ===
RMSD: 0.8338
Reconstruction Loss: 0.9093
KL Divergence (orig->recon): 8.5024
KL Divergence (recon->orig): -0.4640
GIF动画: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/recon_sample_1_gt_field.gif
对比图: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_drugs_20250819_100445_894697/recon/recon_sample_1_gt_field_final.png
